In [1]:
import os

# Define the target directory
target_directory = os.getcwd()  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\azure-ai-agent-services-demo


## Creating Azure AI Agent with SharePoint Integration 

SharePoint as a data source allows you to ground your Azure AI Agents with documents stored in SharePoint securely. You can connect to your SharePoint site, such as `contoso.sharepoint.com/sites/policies`. When a user sends a query, Azure AI Agents will determine if SharePoint should be leveraged or not. If so, it will send the query via the SharePoint tool, which checks if the user has an M365 Copilot license and uses the end user’s identity to retrieve relevant documents they have access to. The scope of retrieval includes all supported documents in this SharePoint site. Lastly, Azure AI Agents will generate responses based on the retrieved information.

With SharePoint integration, we will support **OBO (On-Behalf-Of) authentication**, which allows the SharePoint tool to retrieve relevant documents based on the end user’s identity and access.

**Prerequisites**
- Existing SharePoint site, and ensure developers have access to this SharePoint site.
- Developers and end users must have an M365 Copilot license.
- Ensure your AOAI resource and AI project are in one of the following regions: `westus`, `japaneast`, `francecentral`.

+ **RBAC Roles**
    - To CRUD a SharePoint tool in Azure AI Agent, ensure you have the **AI Developer** role.
    - Ensure end users have the **AI Developer** role to enable OBO authentication.

> Please visit [how-to\setup-sharepoint-azure-ai-agentmd](how-to/setup-sharepoint-azure-ai-agentmd).

**Supported Capabilities**
- Grounding with all supported documents in a SharePoint site.
- Supported document types: PDF, Word, PPT, TXT, .aspx (text data only).
- Automatic indexing of updated documents.
- Responses based on the end user’s document access.
- Connection to a maximum of 1 SharePoint site.

**Known Limitations**
- Retrieval and grounding quality issues, especially with complex formatting, tables, columns, and images.
- Text-sparse file types (e.g., PPT) may return poorer results than text-rich types (e.g., DOCX).
- Grounding with specific libraries or multiple sites is not supported.

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import SharepointTool
from dotenv import load_dotenv
import os
import time
import json
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageTextContent
from azure.core.exceptions import HttpResponseError
from utils.ml_logging import get_logger

logger = get_logger()

# Load environment variables from .env file
load_dotenv()

# The connection string format should be: <HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>
conn_str = os.environ["AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"]

# Create the AIProjectClient using the connection string and explicit credential
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=conn_str,
)

In [3]:
import os
import json
from azure.core.exceptions import HttpResponseError
from utils.utilityfucntions import print_agent_summary

# 1. Set up the model name from environment variable
deployment_name = os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME")
if not deployment_name:
    logger.error("Environment variable 'AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME' is not set.")
    exit(1)

# 2. Initialize the AI Project Client (assumed to be done in a previous cell)
# 2a. Initialize the SharePoint tool using the connection ID.
# For connection setup details, please refer to `how-to/setup-tools-in-ai-foundry.md`
try:
    sharepoint_connection = project_client.connections.get(
        connection_name=os.environ["TOOL_CONNECTION_NAME_SHAREPOINT"]
    )
except KeyError:
    logger.error("Environment variable 'TOOL_CONNECTION_NAME_SHAREPOINT' is not set.")
    exit(1)

conn_id = sharepoint_connection.id
logger.info(f"SharePoint Connection ID: {conn_id}")

# Create a SharePointTool instance with the connection ID
sharepoint = SharepointTool(connection_id=conn_id)

# 3. Create a SharePoint-powered AI Agent
try:
    sharepoint_agent = project_client.agents.create_agent(
        model=deployment_name,
        name="my-sharepoint-assistant",
        description=(
            "A SharePoint-integrated AI assistant designed to search, retrieve, "
            "and summarize documents stored in SharePoint. This assistant helps users "
            "quickly find relevant information while ensuring responses include "
            "document references for validation."
        ),
        instructions=(
            "You are an AI-powered assistant specialized in searching and retrieving "
            "documents from SharePoint. Your primary role is to find the most relevant "
            "documents based on user queries and provide clear, concise summaries.\n\n"
            "**Response Guidelines:**\n"
            "1. Always retrieve the most relevant document(s) from SharePoint.\n"
            "2. Provide a summary of the key information from the retrieved document(s).\n"
            "3. Every response **must include the link** to the original document for reference.\n"
            "4. If no relevant document is found, respond with: 'No matching documents found in SharePoint.'\n"
            "5. Use professional, structured language, and avoid speculation.\n\n"
            "**Example Responses:**\n"
            "✅ 'The requested policy document is available here: [Document Link]. Based on its content, "
            "the key points are...'\n"
            "❌ 'I think this might be what you're looking for...' (Avoid speculation.)"
        ),
        tools=sharepoint.definitions,
        headers={"x-ms-enable-preview": "true"},
        temperature=1,
        top_p=1,
        metadata={
            "use_case": "Enterprise SharePoint Search",
            "data_source": "SharePoint",
            "response_validation": "Must include source link"
        },
    )

    logger.info(f"Created Agent ID: {sharepoint_agent.id}")
    logger.info(f"Agent Metadata: {sharepoint_agent.metadata}")

except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"Error Message: {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"Non-JSON Error Content: {e.response.content}")
    exit(1)

print_agent_summary(sharepoint_agent)


2025-03-19 11:24:49,971 - micro - MainProcess - INFO     SharePoint Connection ID: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-zhuoqunliai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-1959/connections/ContosoAgentDemoSharepoint (614259051.py:<module>:24)
2025-03-19 11:24:53,850 - micro - MainProcess - INFO     Created Agent ID: asst_jtJEpWJTbepxD3XLGrFK6zVF (614259051.py:<module>:66)
2025-03-19 11:24:53,852 - micro - MainProcess - INFO     Agent Metadata: {'use_case': 'Enterprise SharePoint Search', 'data_source': 'SharePoint', 'response_validation': 'Must include source link'} (614259051.py:<module>:67)



=== Agent Creation Summary ===
Agent Name    : my-sharepoint-assistant
Agent ID      : asst_jtJEpWJTbepxD3XLGrFK6zVF
Agent Metadata:
  - use_case: Enterprise SharePoint Search
  - data_source: SharePoint
  - response_validation: Must include source link


In [4]:
try:
    # 1. Create a thread for the conversation
    thread = project_client.agents.create_thread()
    logger.info(f"Created Thread ID: {thread.id}")

    # 2. Create a user message on that thread
    user_message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What are the key features from Dexcom G7 CGM System?"
    )
    logger.info(f"Created User Message ID: {user_message.id}")

    # 3. Create a run to process the conversation
    run = project_client.agents.create_run(
        thread_id=thread.id, agent_id=sharepoint_agent.id
    )
    logger.info("Run created. Polling for status...")

    # 4. Poll until run completes or fails
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(0.5)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        logger.info(f"Current run ID: {run.id}")
        logger.info(f"Current run status: {run.status}")

    logger.info(f"Run finished with status: {run.status}")

    # 5. Retrieve and display the conversation history (oldest to newest)
    conversation_history = project_client.agents.list_messages(thread_id=thread.id)
    logger.info("----- Conversation History -----")
    for msg in conversation_history.data:
        if msg.content and isinstance(msg.content[-1], MessageTextContent):
            logger.info(f"{msg.role.upper()}: {msg.content[-1].text.value}")
except Exception as e:
    logger.error(f"An error occurred during agent processing: {e}")

2025-03-19 11:28:55,554 - micro - MainProcess - INFO     Created Thread ID: thread_Tx06SIbCcvjHTGnk9L3DtMdV (1549044826.py:<module>:4)
2025-03-19 11:28:56,001 - micro - MainProcess - INFO     Created User Message ID: msg_H7bI9S1tN7taCxVYCsgVoN4b (1549044826.py:<module>:12)
2025-03-19 11:28:58,794 - micro - MainProcess - INFO     Run created. Polling for status... (1549044826.py:<module>:18)
2025-03-19 11:28:59,858 - micro - MainProcess - INFO     Current run ID: run_siUPGO6DyMI28DfUV9f4563M (1549044826.py:<module>:24)
2025-03-19 11:28:59,860 - micro - MainProcess - INFO     Current run status: RunStatus.IN_PROGRESS (1549044826.py:<module>:25)
2025-03-19 11:29:00,741 - micro - MainProcess - INFO     Current run ID: run_siUPGO6DyMI28DfUV9f4563M (1549044826.py:<module>:24)
2025-03-19 11:29:00,742 - micro - MainProcess - INFO     Current run status: RunStatus.IN_PROGRESS (1549044826.py:<module>:25)
2025-03-19 11:29:01,689 - micro - MainProcess - INFO     Current run ID: run_siUPGO6DyMI28Df

## Creating Azure AI Agent with Fabric Integration 

SharePoint as a data source allows you to ground your Azure AI Agents with documents stored in SharePoint securely. You can connect to your SharePoint site, such as `contoso.sharepoint.com/sites/policies`. When a user sends a query, Azure AI Agents will determine if SharePoint should be leveraged or not. If so, it will send the query via the SharePoint tool, which checks if the user has an M365 Copilot license and uses the end user’s identity to retrieve relevant documents they have access to. The scope of retrieval includes all supported documents in this SharePoint site. Lastly, Azure AI Agents will generate responses based on the retrieved information.

With SharePoint integration, we will support **OBO (On-Behalf-Of) authentication**, which allows the SharePoint tool to retrieve relevant documents based on the end user’s identity and access.

**Prerequisites**
- Existing SharePoint site, and ensure developers have access to this SharePoint site.
- Developers and end users must have an M365 Copilot license.
- Ensure your AOAI resource and AI project are in one of the following regions: `westus`, `japaneast`, `francecentral`.

+ **RBAC Roles**
    - To CRUD a SharePoint tool in Azure AI Agent, ensure you have the **AI Developer** role.
    - Ensure end users have the **AI Developer** role to enable OBO authentication.

> Please visit [how-to\setup-sharepoint-azure-ai-agentmd](how-to/setup-sharepoint-azure-ai-agentmd).

**Supported Capabilities**
- Grounding with all supported documents in a SharePoint site.
- Supported document types: PDF, Word, PPT, TXT, .aspx (text data only).
- Automatic indexing of updated documents.
- Responses based on the end user’s document access.
- Connection to a maximum of 1 SharePoint site.

**Known Limitations**
- Retrieval and grounding quality issues, especially with complex formatting, tables, columns, and images.
- Text-sparse file types (e.g., PPT) may return poorer results than text-rich types (e.g., DOCX).
- Grounding with specific libraries or multiple sites is not supported.

In [8]:
from azure.ai.projects.models import FabricTool

conn_id = project_client.connections.get(
    connection_name=os.environ["TOOL_CONNECTION_NAME_FABRIC"],
)

# 4. Initialize Fabric Tool
fabric_tool = FabricTool(connection_id=conn_id.id)

# 5. Create the Fabric AI Agent
try:
    fabric_agent = project_client.agents.create_agent(
        model=deployment_name,
        name="fabric-intelligence-agent",
        description=(
            "An AI-powered assistant designed to interact with Microsoft Fabric, "
            "fetch relevant data, analyze insights, and ensure structured responses "
            "with validated references. The agent helps users extract valuable "
            "business intelligence from Fabric data sources."
        ),
        instructions=(
            "You are an AI assistant specialized in retrieving and analyzing data "
            "from Microsoft Fabric. Your primary goal is to help users extract insights "
            "by querying structured and unstructured data sources within Fabric. \n\n"
            "**Response Guidelines:**\n"
            "1. Always retrieve the most relevant dataset(s) from Fabric.\n"
            "2. Provide a structured summary of key findings, ensuring accuracy.\n"
            "3. Every response **must include a link** to the original data source for verification.\n"
            "4. If no relevant data is found, respond with: 'No relevant data found in Fabric.'\n"
            "5. Use precise, professional, and structured language; avoid speculation.\n\n"
            "**Example Responses:**\n"
            "✅ *'The requested sales performance data is available here: [Dataset Link]. Key insights: ...'* \n"
            "❌ *'I think this might be useful...'* (Avoid vague responses.)"
        ),
        tools=fabric_tool.definitions,
        headers={"x-ms-enable-preview": "true"},
        temperature=0.7,
        top_p=1,
        metadata={
            "use_case": "Microsoft Fabric Data Analysis",
            "data_source": "Fabric",
            "response_validation": "Must include source link"
        },
    )

    logger.info(f"Created Agent ID: {fabric_agent.id}")
    logger.info(f"Agent Metadata: {fabric_agent.metadata}")

except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"Error Message: {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"Non-JSON Error Content: {e.response.content}")
    exit(1)

print_agent_summary(fabric_agent)

2025-03-19 11:35:35,525 - micro - MainProcess - INFO     Created Agent ID: asst_fewEhoM0W9x3yKocx09JincX (3394478606.py:<module>:46)
2025-03-19 11:35:35,527 - micro - MainProcess - INFO     Agent Metadata: {'use_case': 'Microsoft Fabric Data Analysis', 'data_source': 'Fabric', 'response_validation': 'Must include source link'} (3394478606.py:<module>:47)



=== Agent Creation Summary ===
Agent Name    : fabric-intelligence-agent
Agent ID      : asst_fewEhoM0W9x3yKocx09JincX
Agent Metadata:
  - use_case: Microsoft Fabric Data Analysis
  - data_source: Fabric
  - response_validation: Must include source link


In [7]:
try:
    # 1. Create a thread for the conversation
    thread = project_client.agents.create_thread()
    logger.info(f"Created Thread ID: {thread.id}")

    # 2. Create a user message on that thread
    user_message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="How does Product A compare to Product B in terms of MARD percentage across different glucose ranges? Use the tools to analyze the data and provide a summary.",
    )
    logger.info(f"Created User Message ID: {user_message.id}")

    # 3. Create a run to process the conversation
    run = project_client.agents.create_run(
        thread_id=thread.id, agent_id=fabric_agent.id
    )
    logger.info("Run created. Polling for status...")

    # 4. Poll until run completes or fails
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(0.5)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        logger.info(f"Current run ID: {run.id}")
        logger.info(f"Current run status: {run.status}")

    logger.info(f"Run finished with status: {run.status}")

    # 5. Retrieve and display the conversation history (oldest to newest)
    conversation_history = project_client.agents.list_messages(thread_id=thread.id)
    logger.info("----- Conversation History -----")
    for msg in conversation_history.data:
        if msg.content and isinstance(msg.content[-1], MessageTextContent):
            logger.info(f"{msg.role.upper()}: {msg.content[-1].text.value}")
except Exception as e:
    logger.error(f"An error occurred during agent processing: {e}")



2025-03-19 11:34:29,805 - micro - MainProcess - INFO     Created Thread ID: thread_IIIZNRIyCyaDL0SvxbJQDj4G (621789375.py:<module>:4)
2025-03-19 11:34:30,183 - micro - MainProcess - INFO     Created User Message ID: msg_02LAgHGC5CQRrbOtIoT49jpG (621789375.py:<module>:12)
2025-03-19 11:34:32,838 - micro - MainProcess - INFO     Run created. Polling for status... (621789375.py:<module>:18)
2025-03-19 11:34:33,841 - micro - MainProcess - INFO     Current run ID: run_rjjCBvDQhGP5BKto4yjSaY8S (621789375.py:<module>:24)
2025-03-19 11:34:33,843 - micro - MainProcess - INFO     Current run status: RunStatus.FAILED (621789375.py:<module>:25)
2025-03-19 11:34:33,845 - micro - MainProcess - INFO     Run finished with status: RunStatus.FAILED (621789375.py:<module>:27)
2025-03-19 11:34:34,254 - micro - MainProcess - INFO     ----- Conversation History ----- (621789375.py:<module>:31)
2025-03-19 11:34:34,258 - micro - MainProcess - INFO     USER: How does Product A compare to Product B in terms of 